In [ ]:
# default_exp property_scorer

In [1]:
from transformers import GPT2Config, GPT2Model
from torch import nn

# Property Scorer

> Score a given input text, property pair. 

The same transformer with a language model head.

In [2]:
class PropertyScorer(GPT2Model):
    def __init__(self):
        config = GPT2Config()
        super().__init__(config)
        self.transformer = GPT2Model(config)
        self.lm_head = nn.Linear(config.n_embd, 1, bias=False)

        self.init_weights()

        # Model parallel
        self.model_parallel = False
        self.device_map = None

# Testing

In [3]:
from kirby.data_manager import DataManager
from kirby.run_params import RunParams
import torch
import numpy as np

In [8]:
x = "Rob is the best"
p = "{Rob : Sux}"

In [9]:
x = x + p

data_manager = DataManager(RunParams())
x = data_manager.tokenizer(x)

<class 'str'> Rob is the best{Rob : Sux}


In [10]:
x

{'input_ids': [14350, 318, 262, 1266, 90, 14350, 1058, 1778, 87, 92], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
x = torch.from_numpy(np.array(x['input_ids']))

In [12]:
model = PropertyScorer()

In [13]:
outputs = model(x)
outputs

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[ 2.1788e-02, -2.8575e-01,  1.0616e+00,  ...,  2.3725e+00,
          9.5130e-02, -4.3660e-01],
        [ 2.7482e-01, -6.2466e-02,  8.4509e-01,  ...,  2.7752e+00,
         -4.5229e-01, -1.3642e+00],
        [-5.1584e-01,  3.1770e-01, -1.9028e-02,  ...,  1.9008e+00,
         -2.0614e-01, -8.7637e-01],
        ...,
        [-4.5148e-01,  1.1863e+00, -1.0356e-01,  ...,  2.6870e+00,
          1.6778e-01, -1.5596e+00],
        [-3.5092e-01,  6.8464e-01,  1.1565e+00,  ...,  2.2763e+00,
          8.4441e-04, -2.4940e+00],
        [ 9.5237e-01,  1.4783e+00,  8.9401e-01,  ...,  1.0049e+00,
          7.6476e-02, -8.2174e-01]], grad_fn=<ViewBackward>), past_key_values=((tensor([[[[ 4.9319e-01,  4.0190e-01, -6.2945e-01,  ...,  1.7219e-01,
            3.4716e-01,  1.0810e+00],
          [ 7.3040e-01,  8.8979e-01,  1.5853e-01,  ...,  4.4759e-01,
            3.6789e-01, -7.9590e-01],
          [ 7.6028e-01,  1.7424e-01,  2.4870e-01,  

In [14]:
hidden_states = outputs[0]
hidden_states.shape

torch.Size([10, 768])

In [15]:
lm_logits = model.lm_head(hidden_states)
lm_logits.shape

torch.Size([10, 1])